## TODO
 - Make a list of important dates (holidays, fast sundays, stake conferences)
 - Add a check for all future-scheduled dates that they don't have any hymns that were sung recently. This would happen if a hymn got changed after the schedule was made.

# Steps to use:
 1. Make sure the history is accurate for the past few weeks
 2. Check if there are any holidays, fast sundays, or special meetings coming up
 3. Run most of the notebook to generate suggestions
 4. Select hymns, enter on Edify
 5. Add selected hymns to history
 6. Add hymns to Gospel Library
 7. Update ward program, if needed, using Gospel Library as reference
 8. Email Ilene Shelton what the hymns are ileneshelton66@gmail.com


In [1]:
import datetime as dt

import numpy as np
import polars as pl

pl.Config().set_tbl_rows(30)
pl.Config().set_fmt_str_lengths(50)

polars.config.Config

In [2]:
def add_last_sung_col(df):
    # Load history df
    history_df = pl.read_csv('history.csv', infer_schema_length=None)
    history_df = history_df.with_columns(
        pl.col("date").str.strptime(pl.Date, "%m/%d/%Y").alias("date"),
    ).drop('name').drop_nulls()
    history_df.head()

    last_sung = history_df.group_by('id').agg(pl.max('date').alias('last_sung'))
    df = df.join(last_sung, on='id', how='left')
    return df


def get_ranking_score(df: pl.DataFrame, type_col: str, popularity_weight=0.5, noise=0.0) -> pl.Series:
    score_df = df.filter((pl.col(type_col) == 1) & (pl.col('flagged').fill_null(0) != 1))
    weeks_since_fill_value = 100
    # Add days since last sung column
    score_df = score_df.with_columns(
        ((pl.lit(dt.datetime.today()) - pl.col('last_sung')).dt.total_days()//7)
        .fill_null(weeks_since_fill_value)
        .alias('weeks_since')
    )

    midpoint = 7
    slope = 2

    def popularity_func(x): return np.round(
        1 / (1 + np.exp(-slope * (x - midpoint))), 2)
    weeks_cutoff = 8
    score_df = score_df.with_columns((pl.col('popularity') + pl.col('popularity_adjustment').fill_null(0)).alias('adj_popularity'))
    score_df = score_df.with_columns(
        pl.col('adj_popularity').map_elements(popularity_func).alias('popularity_score'),
        pl.when(pl.col('weeks_since') > weeks_cutoff)
        .then((pl.col('weeks_since') / (weeks_since_fill_value/2)).round(3))
        .otherwise(-100).alias('weeks_since_score'),
        pl.Series(np.round(np.random.uniform(-noise, noise,
                  len(score_df)), 2)).alias('noise'),
    ).with_columns(
        (pl.col('popularity_score') * popularity_weight + pl.col('weeks_since_score')
         * (1 - popularity_weight)).alias('score') + pl.col('noise')
    )

    score_df = score_df.select([
        'id', 'name', 'length', 'adj_popularity', 'weeks_since', 'popularity_score', 'weeks_since_score', 'score'
    ]).sort('score', descending=True)

    return score_df

In [3]:
df = pl.read_csv("hymns.csv")
df = add_last_sung_col(df)

## Sacrament Hymn

In [4]:
sacrament_ranking = get_ranking_score(df, 'is_sacrament', noise=0.1)
display(sacrament_ranking.head(10))

id,name,length,adj_popularity,weeks_since,popularity_score,weeks_since_score,score
i64,str,str,i64,i64,f64,f64,f64
189,"""O Thou, Before the World Began""","""3:06""",6,100,0.12,2.0,1.12
192,"""He Died! The Great Redeemer Died""","""3:05""",10,38,1.0,0.76,0.9
195,"""How Great the Wisdom and the Love""","""3:57""",10,39,1.0,0.78,0.89
187,"""God Loved Us, So He Sent His Son""","""1:46""",10,33,1.0,0.66,0.85
174,"""While of These Emblems We Partake (AEOLIAN)""","""3:05""",10,35,1.0,0.7,0.84
179,"""Again, Our Dear Redeeming Lord""","""2:36""",9,27,0.98,0.54,0.83
191,"""Behold the Great Redeemer Die""","""3:19""",10,25,1.0,0.5,0.83
170,"""God, Our Father, Hear Us Pray""","""2:21""",10,32,1.0,0.64,0.79
190,"""In Memory of the Crucified""","""2:32""",10,31,1.0,0.62,0.79


## General Hymns

In [5]:
general_ranking = get_ranking_score(df, 'is_general', noise=0.15, popularity_weight=0.7)
# display(general_ranking.head(30))
display(general_ranking.filter(pl.col('name').str.contains('Now')))

id,name,length,adj_popularity,weeks_since,popularity_score,weeks_since_score,score
i64,str,str,i64,i64,f64,f64,f64
156,"""Sing We Now at Parting""","""2:47""",8,36,0.88,0.72,0.922
3,"""Now Let Us Rejoice""","""3:29""",10,23,1.0,0.46,0.888
95,"""Now Thank We All Our God""","""2:18""",6,100,0.12,2.0,0.724
91,"""Father, Thy Children to Thee Now Raise""","""1:51""",5,100,0.02,2.0,0.694
162,"""Lord, We Come before Thee Now""","""2:38""",3,100,0.0,2.0,0.67
160,"""Softly Now the Light of Day""","""0:51""",2,100,0.0,2.0,0.64
25,"""Now We'll Sing with One Accord""","""3:28""",5,100,0.02,2.0,0.624
159,"""Now the Day Is Over""","""1:36""",3,100,0.0,2.0,0.46


# Add to History

In [6]:
raise KeyboardInterrupt("This is here to stop you from running the whole notebook.")

KeyboardInterrupt: This is here to stop you from running the whole notebook.

In [9]:
def name_from_id(id):
    name_row = df.filter(pl.col("id") == id).select("name")
    if len(name_row) != 1:
        raise ValueError(f"Expected 1 row, got {len(name_row)}")
    return name_row["name"].item()


def format_for_csv(date, opening, sacrament, intermediate, closing):
    str_list = []
    if opening:
        str_list.append(
            f"{date.strftime('%m/%d/%Y')},opening,{opening},\"{name_from_id(opening)}\""
        )
    if sacrament:
        str_list.append(
            f"{date.strftime('%m/%d/%Y')},sacrament,{sacrament},\"{name_from_id(sacrament)}\""
        )
    if intermediate:
        str_list.append(
            f"{date.strftime('%m/%d/%Y')},intermediate,{intermediate},\"{name_from_id(intermediate)}\""
        )
    if closing:
        str_list.append(
            f"{date.strftime('%m/%d/%Y')},closing,{closing},\"{name_from_id(closing)}\""
        )
    csv_str = "\n".join(str_list)
    return csv_str

In [31]:
date = dt.date(2024, 1, 28)
opening = 30
sacrament = 174
intermediate = 249
closing = 98

In [32]:
csv_str = format_for_csv(date, opening, sacrament, intermediate, closing)
print("Does this look right?")
print(csv_str)

Does this look right?
01/28/2024,opening,30,"Come, Come, Ye Saints"
01/28/2024,sacrament,174,"While of These Emblems We Partake (AEOLIAN)"
01/28/2024,intermediate,249,"Called to Serve"
01/28/2024,closing,98,"I Need Thee Every Hour"


In [33]:
# Append to history csv
with open('history.csv', 'a') as f:
    f.write(csv_str)
    f.write("\n")